# *<foo><font color='orange'> SONG ELEMENT PREDICTION: MODELLING THE "MILLENIAL WHOOP"</font></foo>*

## INTRODUCTION

PURPOSE: 

    In this project we strive to understand the elements of modern pop music that affect song popularity. In this project we will investigate the occurance of the "Millenial Whoop." 

BACKGROUND: 

    [Insert dude's name] identified a common 3-5 chord progression used commonly in modern pop music. The claim is that this 3-5 chord progression known as the "Millenial Whoop" is a satisfying song element that helps ensure song likability. Song likability reduces the financial risk incured by a record company upon release of a new song. For more on [insert dude's name]'s work, see here [insert link].
    
APPROACH:

    For the sake of of this project we will try to a) train a model to identify the "Millenial Whoop" in a song, and b) see if there exists a relationship between the rate of occurance of a song "Whoop" and the popularity of that song, as measured using Spotify's algorithm. 
    
TOOLS:

     Tools used include the Spotify Web API for song attributes and audio analysis. We will also reconstruct a dataframe to represent the frequencies on a [octave scale?]. Lastly, we will utilize web scraping to collect the songs and their time information that are known to contain the "Millenial Whoop." These will all be used to explore, hypothesis, train and test our model. 

## DATA COLLECTION

In [46]:
#imports

%matplotlib inline 

#graphical imports
from IPython.display import Image
import matplotlib as mlp
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import mpl_toolkits.mplot3d as m3d

#numerical, os and dataframe imports
import numpy as np
import os
import pandas as pd

#data modelling imports 
#from sklearn import cross_validation  -> deprecated 
import sklearn
from sklearn import datasets
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing 
from sklearn import model_selection
from sklearn import feature_selection

#stats imports
from scipy import stats
from scipy import stats, special

#file imports
import json
import ijson
from pandas.io.json import json_normalize

**Begin by exploring format of raw Spotify JSON Audio Analysis file, for key 'Segments'**

In [26]:
file = 'C:\\Users\\Jnfr\\Projects\\SpotifyAnalyticsApp\\amy_winehouse_youknow_analysis.json'

# read file by key 'segments' 
with open(file, 'r') as json_data:
    segments = ijson.items(json_data, 'segments')
    segCols = list(segments)

#store normalized segment file in pandas dataframe
df = pd.DataFrame.from_dict(json_normalize(segCols[0]))


In [27]:
df

,confidence,duration,loudness_end,loudness_max,loudness_max_time,loudness_start,pitches,start,timbre
0,0,0.15859,NaN,-60,0,-60,"[0.085, 0.569, 0.561, 0.550, 0.977, 1, 0.545, ...",0,"[0, 171.130, 9.469, -28.480, 57.491, -50.067, ..."
1,1,0.32685,NaN,-6.742,0.02952,-60,"[0.309, 1, 0.409, 0.097, 0.117, 0.148, 0.088, ...",0.15859,"[37.162, -8.502, -141.823, 41.399, 29.980, 213..."
2,0.955,0.27578,NaN,-18.090,0.01485,-32.812,"[1, 0.715, 0.109, 0.153, 0.235, 0.663, 0.381, ...",0.48544,"[32.633, 112.576, -82.864, 101.897, -18.587, -..."
3,1,0.31401,NaN,-0.448,0.04120,-37.522,"[0.446, 0.558, 0.251, 0.195, 0.323, 0.317, 0.5...",0.76122,"[43.321, 84.085, -83.058, 83.784, 30.820, 176...."
4,0.851,0.09134,NaN,-18.683,0.01001,-29.840,"[0.846, 1, 0.976, 0.979, 0.634, 0.634, 0.217, ...",1.07524,"[34.789, 142.319, -10.702, 55.957, -31.650, -8..."
5,1,0.16989,NaN,-10.483,0.06243,-30.588,"[0.237, 0.305, 0.245, 0.280, 0.343, 0.460, 0.4...",1.16658,"[39.343, 139.490, 21.473, -120.363, 1.552, 65...."
6,0.818,0.19646,NaN,-9.208,0.04012,-21.949,"[0.664, 1, 0.505, 0.190, 0.329, 0.416, 0.328, ...",1.33646,"[42.291, 109.967, -102.485, 35.953, -22.875, 3..."
7,0.808,0.09737,NaN,-17.388,0.01641,-28.971,"[0.895, 1, 0.164, 0.205, 0.229, 0.266, 0.517, ...",1.53293,"[37.888, 98.539, -46.944, 0.080, -20.645, -72...."
8,1,0.27837,NaN,-8.548,0.00484,-29.149,"[0.672, 1, 0.365, 0.119, 0.131, 0.126, 0.058, ...",1.63029,"[42.262, -49.009, -134.198, 86.499, -17.903, -..."
9,1,0.30313,NaN,-1.362,0.04262,-31.169,"[0.408, 0.329, 0.168, 0.166, 0.353, 0.385, 0.5...",1.90866,"[44.573, 74.752, -79.899, 117.529, 34.865, 126..."


In [107]:
#plt.plot(df);
#data_dict = dict(zip((str(df['start'])), (df['pitches'])))
#data_dict = dict((str(pitch),pitch) for pitch in df['pitches'])
#py.plot(data_dict, filename='basic-scatter')
#scatter(df['pitches'], df['start']);

#df.plot(x='start', y='pitches')

#plt.scatter(df['start'], df['pitches'])
#plt.show()
#df.columns
dataframe = pd.DataFrame(df['pitches'])
dataframe

#df.T.plot( kind='bar') # or df.T.plot.bar()
#plt.show()

,pitches
0,"[0.085, 0.569, 0.561, 0.550, 0.977, 1, 0.545, ..."
1,"[0.309, 1, 0.409, 0.097, 0.117, 0.148, 0.088, ..."
2,"[1, 0.715, 0.109, 0.153, 0.235, 0.663, 0.381, ..."
3,"[0.446, 0.558, 0.251, 0.195, 0.323, 0.317, 0.5..."
4,"[0.846, 1, 0.976, 0.979, 0.634, 0.634, 0.217, ..."
5,"[0.237, 0.305, 0.245, 0.280, 0.343, 0.460, 0.4..."
6,"[0.664, 1, 0.505, 0.190, 0.329, 0.416, 0.328, ..."
7,"[0.895, 1, 0.164, 0.205, 0.229, 0.266, 0.517, ..."
8,"[0.672, 1, 0.365, 0.119, 0.131, 0.126, 0.058, ..."
9,"[0.408, 0.329, 0.168, 0.166, 0.353, 0.385, 0.5..."


In [100]:
with open(file, 'r') as json_data2:
    track = ijson.items(json_data2, 'track')
    trackCols = list(track)
    
df2 = pd.DataFrame.from_dict(json_normalize(trackCols[0]))

In [101]:
df2

,analysis_channels,analysis_sample_rate,code_version,codestring,duration,echoprint_version,echoprintstring,end_of_fade_in,key,key_confidence,...,rhythmstring,sample_md5,start_of_fade_out,synch_version,synchstring,tempo,tempo_confidence,time_signature,time_signature_confidence,window_seconds
0,1,22050,3.15,eJxNnAmWxTaOBK-iI3Bf7n8xRyR_2Z5-022x9CkSBBKJha...,257.18667,4.12,eJzd_WuO9TpyRYt2SRKfag5Fkf1vwh2DacD2wRHXj-8UDF...,0.15859,2,0.175,...,eJxdXIeRJDkMS6VDkKNM_om9YKjZ-6rbut0x3TIkCIJU11...,,246.57270,1,eJxdWQuW5DgIu0qOEMDf-19skQTpmn3T05VybMxHCOx2f-...,103.340,0.545,4,1,0


In [6]:
with open(file, 'r') as json_data3:
    sections = ijson.items(json_data3, 'sections')
    secCols = list(sections)
    
df3 = pd.DataFrame.from_dict(json_normalize(secCols[0]))

In [7]:
df3

,confidence,duration,key,key_confidence,loudness,mode,mode_confidence,start,tempo,tempo_confidence,time_signature,time_signature_confidence
0,1,14.93769,0,0.116,-6.211,1,0.406,0,103.875,0.931,4,1
1,0.845,22.58094,2,0,-3.422,0,0,14.93769,103.641,0.548,4,1
2,0.542,12.42644,4,0,-1.583,1,0,37.51863,104.490,0.526,4,1
3,0.232,20.50631,2,0.052,-3.944,1,0.447,49.94507,102.552,0.718,4,1
4,0.485,28.19845,5,0.189,-3.812,0,0.462,70.45138,102.368,0.672,4,1
5,0.519,12.80046,0,0.683,-2.729,1,0.563,98.64982,103.737,0.532,4,1
6,0.358,20.44131,1,0,-3.254,0,0,111.45029,102.355,0.638,4,1
7,0.250,17.83745,9,0,-2.960,1,0,131.89159,104.721,0.621,4,1
8,0.566,7.00472,3,0,-1.187,1,0,149.72904,104.100,0.356,4,1
9,0.463,16.20283,2,0,-3.298,0,0,156.73376,103.591,0.561,4,1


In [8]:
with open(file, 'r') as json_data3:
    beats = ijson.items(json_data3, 'beats')
    beatCols = list(beats)
    
df4 = pd.DataFrame.from_dict(json_normalize(beatCols[0]))

In [9]:
df4

,confidence,duration,start
0,1,0.58429,0.49871
1,0.961,0.57670,1.08300
2,0.838,0.57780,1.65970
3,0.967,0.57734,2.23750
4,0.961,0.57714,2.81483
5,0.936,0.58004,3.39197
6,0.958,0.57996,3.97202
7,0.943,0.57998,4.55198
8,0.950,0.58207,5.13196
9,0.941,0.58260,5.71403


### CREATE FREQUENCY DATAFRAME

In [10]:
data = {'octave_num': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1','1','1','1','1','1','1','1','1','1',
                       '2', '2','2','2','2','2','2','2','2','2','2','2','3','3','3','3','3','3','3','3','3','3','3','3','4','4','4','4','4','4','4','4','4','4','4','4',
                       '5','5','5','5','5','5','5','5','5','5','5','5','6','6','6','6','6','6','6','6','6','6','6','6','7','7','7','7','7','7','7','7','7','7','7','7',
                       '8','8','8','8','8','8','8','8','8','8','8','8'], 'note': [ 'C', 'C#','D', 'Eb', 'E','F', 'F#', 'G', 'G#', 'A', 'Bb', 'B', 'C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B',
                       'C', 'C#','D','Eb','E','F','F#', 'G', 'G#', 'A', 'Bb', 'B','C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B',
                       'C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B','C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B',
                       'C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B', 'C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B',
                       'C', 'C#','D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B'],'frequency': [16.35,17.32,18.35,19.45,20.60,21.83,23.12,24.50,25.96,27.50,29.14,30.87,32.70,34.65,36.71,38.89,41.20,43.65,46.25,49.00,51.91,55.00,58.27,61.74,65.41,69.30,73.42,77.78,82.41,87.31,92.50,98.00,103.8,110.0,116.5,123.5,130.8,138.6,146.8,155.6,164.8,174.6,185.0,196.0,207.7,220.0,233.1,246.9,261.6,277.2,293.7,311.1,329.6,349.2,370.0,392.0,415.3,440.0,466.2,493.9,523.3,554.4,587.3,622.3,659.3,698.5,740.0,784.0,830.6,880.0,932.3,987.8,1047,1109,1175,1245,1319,1397,1480,1568,1661,1760,1865,1976,2093,2217,2349,2489,2637,2794,2960,3136,3322,3520,3729,3951,4186,4435,4699,4978,5274,5588,5920,6272,6645,7040,7459,7902]}


df_frequency = pd.DataFrame(data)
df_frequency.head()

,frequency,note,octave_num
0,16.35,C,0
1,17.32,C#,0
2,18.35,D,0
3,19.45,Eb,0
4,20.60,E,0


In [11]:
#load whoop data
import csv
from io import StringIO
import re
import pandas as pd

file_whoop = "c:\\Users\\Jnfr\\Projects\\SpotifyAnalyticsApp\\whoops.csv"

# get the data as a python string
with open (file_whoop, "r") as myfile:
    data=myfile.read()

# munge in python - get rid of the garbage in the input (lots of xff bytes)
data = re.sub(r'[^,a-zA-Z0-9_\.;:\n]', '', data) # get rid of the rubbish
#data = data + '\n' # the very last one is missing?
#data = re.sub(r';\n', r'\n', data) # last ; separator on line is problematic

# now let's suck into a pandas DataFrame
columns = ['artist', 'song_name', 'year', 'whoop_time', 'youtube_url']
df = pd.read_csv(StringIO(data), index_col=None, header=0, sep=',')
df.columns = columns
'''with open(file_whoop, 'r') as f:
    readCSV = csv.reader(f, delimiter=',')
    print(readCSV)
    #df = pd.DataFrame(object) '''


#df = pd.read_csv(file_whoop, header=0,encoding='iso-8859-1', error_bad_lines=False)
#df = pd.read_csv(StringIO(file_whoop), encoding = "cp1252")
df
#list = open(file_whoop, 'r')
#df_whoops = pd.DataFrame([list.read()])
#print(df_whoops)


,artist,song_name,year,whoop_time,youtube_url
0,BOY,LittleNumbers,2011,1:04,https:www.youtube.comwatchvzsyjS_vJfkw
1,Stonefox,AllIWant,2013,2:02,https:www.youtube.comwatchvwCfNRIGA5_A
2,DemiLovato,IReallyDontCare,2013,1:00,https:www.youtube.comwatchvOJGUbwVMBeA
3,PavoPavo,RanRanRun,2016,0:51,https:www.youtube.comwatchvK9P5p94x2Os
4,Skechers,TwinkleToes,2016,0:00,https:www.youtube.comwatchvXdFoCWzQHhI
5,FrankOcean,Ivy,2016,2:53,https:www.youtube.comwatchv8ndyzG1JCxE
6,Dagny,Backbeat,2016,0:00,https:www.youtube.comwatchv7K0y1RT6M68
7,AURORA,RunningWiththeWolves,2016,1:11,https:www.youtube.comwatchv06ht9MyJLT4
8,AURORA,Conqueror,2016,0:23,https:www.youtube.comwatchvdmVAmlpbnD4
9,BerlinAfterMidnight,AllNightLong,2016,0:52,https:www.youtube.comwatchvNHAiBD2dHU


### CREATE WHOOP DATAFRAME

# DATA WRANGLING

# DATA EXPLORATION

# HYPOTHESIS

# MODEL TRAINING

# MODEL TESTING

# CONCLUSION

# DISCUSSION